## Попытка в использование компоновщика

Сейчас попробуем использовать шаблон "компоновщик" в... Анализе данных. Но сперва немного импортов:

In [2]:
import warnings
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Что по данным?
Возьмем их [из этого соревнования](https://www.kaggle.com/c/mercedes-benz-greener-manufacturing) ([классная статья об этом](https://habr.com/ru/company/ods/blog/336168/)). И теперь посмотрим, как можно было сделать решение. И заодно посмотрим на скор на трейн-тест сплите (использовать что-то другое для валидации лень)

In [6]:
data = pd.read_csv('train.csv')
data.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [7]:
y = data['y']
X = data.drop('y', axis=1).select_dtypes(include=[np.number])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
model = xgb.XGBRegressor()
model.fit(X_train, y_train)
r2_score(y_test, model.predict(X_test))

[11:37:14] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.5700646104533826

# Окей, но это для слабаков
Мы то тут хотим блеснуть знаниями шаблонов проектирования, поэтому будем писать мега-суперансамбль, просто потому что можем. Для этого возьмем и разобьем фичи на две группы, а то иначе скучно

In [8]:
X_first_half_train = X_train.iloc[:, 0:181]
X_second_half_train = X_train.iloc[:, 181:]
X_first_half_test = X_test.iloc[:, 0:181]
X_second_half_test = X_test.iloc[:, 181:]

Теперь опишем, собственно, сам шаблон

In [11]:
from __future__ import annotations
from abc import ABC, abstractmethod


# здесь совсем ничего необычного
class Component(ABC):

    @property
    def parent(self) -> Component:
        return self._parent
    @parent.setter
    def parent(self, parent: Component):
        self._parent = parent

    def add(self, component: Component) -> None:
        pass
    
    def remove(self, component: Component) -> None:
        pass
    
    @abstractmethod
    def operation(self) -> str:
        pass


# теперь наш лист выглядит так
class subModel(Component):
    
    #В лист мы будем передавать модель, которую собираемся использовать тут
    def __init__(self, model):
        self.model = model
    
    # в качестве операции будем обучать модель, а затем предсказывать результат
    def operation(self, X_train, y_train, X_test):
        self.model.fit(X_train, y_train)
        return self.model.predict(X_test)


# А это контейнер
class Ensemble(Component):

    # в каждую ветку дадим возможность передавать свой датасет
    def __init__(self, *params) -> None:
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self._children: List[Component] = []

    def add(self, component: Component) -> None:
        self._children.append(component)
        component.parent = self
        
    def remove(self, component: Component) -> None:
        self._children.remove(component)
        component.parent = None

    # здесь операция будет просто усреднять результаты дочерних элементов, потому что ничего прикольнее я придумать не смог
    def operation(self, *params):
        results = np.array([])
        n = 0
        for child in self._children:
            n += 1
            if len(results) > 0:
                results += child.operation(self.X_train, self.y_train, self.X_test)
            else:
                results = child.operation(self.X_train, self.y_train, self.X_test)
        return results / n

# А теперь давайте тестировать это чудо

Сперва попробуем взять просто бустинг и случайный лес и усреднить это на всех данных

In [13]:
result_model = Ensemble(X_train, y_train, X_test)
model1 = subModel(RandomForestRegressor())
model2 = subModel(xgb.XGBRegressor())
result_model.add(model1)
result_model.add(model2)
# смотрим на результат
r2_score(y_test, result_model.operation())

[11:54:17] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.5284631285788601

А теперь сделаем более сложную структуру с использованием разделенного датасета

In [14]:
# первая ветка будет пользоваться только первой частью датасета, а обучать мы будем простые линейные модели
branch1 = Ensemble(X_first_half_train, y_train, X_first_half_test)
model1 = subModel(LinearRegression())
model2 = subModel(Ridge(alpha=1.0))
branch1.add(model1)
branch1.add(model2)

# второй ветке достается вторая часть данных, а моделями послужат: бустинг и рандомный лес
branch2 = Ensemble(X_second_half_train, y_train, X_second_half_test)
model3 = subModel(RandomForestRegressor())
model4 = subModel(xgb.XGBRegressor())
branch2.add(model3)
branch2.add(model4)

# а теперь соберем это все вместе
result_model = Ensemble()
result_model.add(branch1)
result_model.add(branch2)

# посмотрим на результат
r2_score(y_test, result_model.operation())

[12:00:17] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.5480029121304864

# Что в итоге?
- На нашей валидирующей выборке этот ансамбль оказался слабее стандартного бустинга. При желании можно попробовать поиграться с другими моделями, везде подобрать параметры, заменить простое усреднение чем-то более интересным, еще что-то сделать с данными и протестировать все это уже на лидерборде
- Все это выглядит слишком громоздко и непонятно для задачи с kaggle, ведь все то же самое делается в несколько раз короче
- Если довольно плотно поработать над этой идеей, то можно сделать простенькую библиотечку для бэггинга/стекинга моделей (для интересующихся: [читать тут](https://neurohive.io/ru/osnovy-data-science/ansamblevye-metody-begging-busting-i-steking/))